# CIL: Denoising Postprocessing based on Ising Model

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd /content/drive/My\ Drive/CIL

/content/drive/My Drive/CIL


In [0]:
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import convolve
from scipy.signal import fftconvolve
from scipy.special import logsumexp
from sklearn.metrics.cluster import adjusted_mutual_info_score
import sklearn as skl
import sklearn

import time
import warnings

warnings.filterwarnings('ignore', category=FutureWarning)

import models.unet as unet
import preproc.get_data as data
import postproc.save_submission as subm 
import metrics.visual as vis
from metrics.visual import plot, plot_training_history
import metrics.numerical
from skimage.io import imshow

In [4]:
%tensorflow_version 2.x
import tensorflow as tf
GPU = tf.test.gpu_device_name()
if GPU != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(GPU))

Found GPU at: /device:GPU:0


In [0]:
# As suggested on Piazza
def softmax(x, axis=None):
    return np.exp(x - logsumexp(x, axis=axis, keepdims=True))

In [0]:
import importlib
importlib.reload(subm)

<module 'postproc.save_submission' from '/content/drive/My Drive/CIL/postproc/save_submission.py'>

In [0]:
x_train, x_val, y_train, y_val = data.get_training_validation_data(rotate=True, normalise_x=True)

In [0]:
x_test, x_test_names = data.get_test_data(normalise_x=True)

In [0]:
USE_CLASS_WEIGHTS = True

class_weights = None
if USE_CLASS_WEIGHTS:
    flattened_y = []
    for a in y:
        flattened_y.extend(a.flatten())
    flattened_y = np.asarray(flattened_y)
    flattened_y.shape
    class_weights = sklearn.utils.class_weight.compute_class_weight('balanced',
                                                                    np.unique(flattened_y),
                                                                    flattened_y)

In [0]:
with tf.device(GPU):
  model = unet.get_model(None, None, 3, do_compile=True)

In [0]:
with tf.device(GPU):
  herstory = unet.fit(x_train, y_train, model, epochs=10, validation_data=(x_val, y_val), class_weight=None)
  vis.plot_training_history(herstory)

In [0]:
with tf.device(GPU):
  model = unet.load('checkpoints/unet_crossentr.h5')

In [0]:
with tf.device(GPU):
  preds = model.predict(x_test)

**Implementation of the Ising Model Stuff**

In [0]:
def energy(img_estimate, img_noisy_observation, H, Beta, Eta):
    """Compute the energy for given estimate 'img_estimate'
    with respect to the  observation 'img_noisy_observation'.

    Args:
            img_estimate (np.ndarray): estimated image matrix
            img_noisy_observation (np.ndarray): noisy image matrix

    Returns:
            energy (float): energy of the estimate given observation
    """
    kernel = np.asarray([[1,1,1],[1,0,1],[1,1,1]])
    neigh_sum = convolve(img_estimate, kernel, mode="constant", cval=0)
    return - np.sum(H * img_estimate + \
                    Eta * img_estimate * img_noisy_observation + \
                    Beta * img_estimate * neigh_sum)

Estimating loss via mean squared error

In [0]:
def mse_loss(img_estimate, img_original):
    """Computing mean squared error loss

    Args:
            img_estimate (np.ndarray): estimated image matrix
            img_original (np.ndarray): original image matrix

    Returns:
            mse (float): mean squared error between two matrices
    """
    err = np.sum((img_estimate.astype("float") - img_original.astype("float")) ** 2)
    err /= float(img_estimate.shape[0] * img_estimate.shape[1])
    return err

Auxiliary function for the local energy computation of the implemented MCMC method.

In [0]:
def pixel_energy(img_estimate, img_noisy_observation, i, j):
    """Compute the energy localized around a pixel (i,j)

    Args:
            img_estimate (np.ndarray): estimated image matrix
            img_noisy_observation (np.ndarray): noisy image matrix
            i,j (double, double): pixel coordinates

    Returns:
            energy (float): local energy around given pixel
    """
    return - ( H*img_estimate[i,j]+\
               Beta*img_estimate[i,j]*np.sum(img_estimate[i-1:i+2,j-1:j+2])+\
               Eta*img_estimate[i,j]*img_noisy_observation[i,j])

In [0]:
def meanfield(img_noisy_observation, img_original, epochs, T=1.0, H=0., Beta=1., Eta=1.):
    """Do the meanfield approximation to estimate the reconstruction.

    Args:
            img_noisy_observation (np.ndarray): noisy image matrix
            img_original (np.ndarray): original image matrix
                - Note: we use it only so we can keep track of the
                  MSE drop i.e. for the plotting purposes.
            epochs (int): number of iterations

    Returns:
            img_estimate (np.ndarray): return reconstucted estimate of the original
            energies (np.ndarray): energies recorded in arbitary moments during 
                                      the sampling process, for plotting purposes
            losses (np.ndarray): MSE loss wrt original image recorded in arbitary 
                                 moments during the sampling process, for plotting purposes
    """
    
    # Your code should replace the following code, which is given as a placeholder
    # and demonstrates how the returned values are used in the rest of the code
    
    N = img_noisy_observation.shape[0] * img_noisy_observation.shape[1]
    Nh = img_noisy_observation.shape[0]
    Nw = img_noisy_observation.shape[1]
    
    energies = losses = None

    if not img_original is None:
      E = energy(img_noisy_observation, img_noisy_observation, H, Beta, Eta)
      L = mse_loss(img_noisy_observation, img_original)
      energies = [E]
      losses = [L]
    
    c_bar = img_noisy_observation.copy()
    
    for e in range(epochs):
        a = (1/T) * (Beta * convolve(c_bar, np.asarray([[1,1,1], [1,0,1], [1,1,1]]), mode='constant', cval=0) + Eta * img_noisy_observation + 2*H)
        c_bar = np.tanh(a)
        
        if not img_original is None:
          energies.append(energy(c_bar, img_noisy_observation, H, Beta, Eta))
          losses.append(mse_loss(c_bar, img_original))
        
    return np.reshape(c_bar, img_noisy_observation.shape), np.asarray(energies), np.asarray(losses)

In [0]:
with tf.device(GPU):
  pred_v = model.predict(x_val)

In [0]:
def denoise_ising(predictions, epochs=10, T=1, Beta=0.1, Eta=0.1):
  denoised = np.zeros((predictions.shape[0], predictions.shape[1], predictions.shape[2]))
  for i, pred in enumerate(predictions):
    d, _, _ = meanfield(pred[:,:,0], None, epochs=epochs, T=T, Beta=Beta, Eta=Eta)
    denoised[i] = d
  return denoised

In [0]:
pred_v_denoised = denoise_ising(pred_v > 0.5, Beta=0.1)

In [0]:
for i in np.random.choice(len(pred_v), 20, replace=False):
  plot([x_val[i], y_val[i]])
  plot([pred_v[i, :, :, 0], pred_v_denoised[i]])

In [0]:
from metrics.numerical import accuracy

In [0]:
acc_v = accuracy(y_val, pred_v > 0.5)
acc_dv = accuracy(y_val, pred_v_denoised > 0.5)

In [0]:
acc_v, acc_dv

(0.966638203125, 0.793401875)

In [0]:
pred_test = model.predict(x_test)

In [0]:
pred_test_d = denoise_ising(pred_test > 0.5, Beta=0.1)

In [0]:
subm.save_predictions(pred_test_d, x_test_names, 'ising_denoising', allow_overwrite=True)

93/94, test_93.png

## Conditional Random Field

Reference: Implementation based on https://github.com/Jumpst3r/printed-hw-segmentation/blob/master/FCN/post.py and the accompanying BSc Thesis. It is itself adapted from https://github.com/lucasb-eyer/pydensecrf/tree/master/pydensecrf


In [12]:
!pip install pydensecrf

     |████████████████████████████████| 1.0MB 6.1MB/s 
  Created wheel for pydensecrf: filename=pydensecrf-1.0rc3-cp36-cp36m-linux_x86_64.whl size=2153655 sha256=53809463c741bd8efc3229cdbd912a871b0adb38894922690fd5f8e52356c87c
  Stored in directory: /root/.cache/pip/wheels/92/6f/ec/5c49c25de8c42c872de50ff53582ba3ead850ce52a81e73ac7
Successfully built pydensecrf


In [13]:
!pip install scikit-image

In [0]:
import pydensecrf.densecrf as dcrf
from pydensecrf.utils import (unary_from_softmax)
from skimage import img_as_ubyte

In [0]:
def crf(original, pred, steps=3):
  annotated = np.asarray([1 - pred, pred])

  original = img_as_ubyte(original)
  # annotated = np.moveaxis(annotated, -1, 0)
  annotated = annotated.copy(order='C')

  d = dcrf.DenseCRF2D(original.shape[1], original.shape[0], 2)
  U = unary_from_softmax(annotated)
  d.setUnaryEnergy(U)

  d.addPairwiseGaussian(sxy=(5,5), compat=10, kernel=dcrf.DIAG_KERNEL, 
                        normalization=dcrf.NORMALIZE_SYMMETRIC)
  d.addPairwiseBilateral(sxy=(80,80), srgb=(13,13,13), rgbim=original, compat=0,
                        kernel=dcrf.DIAG_KERNEL, 
                        normalization=dcrf.NORMALIZE_SYMMETRIC)

  Q = d.inference(steps)
  MAP = np.argmax(Q, axis=0).reshape(original.shape[0], original.shape[1])

  return MAP



In [0]:
pred_crf = [crf(x_val[i], pred_v[i], steps=10) for i in range(len(x_val))]
pred_crf = np.asarray(pred_crf)

In [74]:
print(accuracy(y_val, pred_v > 0.5))
print(accuracy(y_val, pred_crf))

0.968616171875
0.96644109375
